На основе ТЕХНИЧЕСКОГО РЕГЛАМЕНТА ТАМОЖЕННОГО СОЮЗА "О БЕЗОПАСНОСТИ ЖЕЛЕЗНОДОРОЖНОГО ПОДВИЖНОГО СОСТАВА" сделайте нейро-консультанта, отвечающего на вопросы по документу.
Проработайте промпт самостоятельно. Нейро-консультант не должен отвечать на вопросы, не касающиеся документа (необходимо предусмотреть это в промпте).
Поэкспериментируйте со сплиттерами, найдите оптимальный вариант.
Сделайте нейро-консультанта, проверьте его работу на самостоятельно-сформулированных вопросах (достаточно 2-3)

Ссылка на регламент: https://docs.google.com/document/d/1YhUEX9fZDNTeE3eJ-yXskxZG46LsTRYvXjZ9Ij-t3Gw

In [1]:
# !pip install faiss-cpu langchain==0.0.271 openai==0.28.1 tiktokenm

In [2]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import re
import requests
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.text_splitter import MarkdownHeaderTextSplitter
# import matplotlib.pyplot as plt
import tiktoken
from langchain.docstore.document import Document

from config import API_TOKEN, PROMT
from task3.errors_processing import errors_handler


# openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = API_TOKEN
openai.api_key = API_TOKEN

In [3]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [4]:
# URL для документа с регламентом:
URL = 'https://docs.google.com/document/d/1YhUEX9fZDNTeE3eJ-yXskxZG46LsTRYvXjZ9Ij-t3Gw'

# Документ с регламентом:
document = load_document_text(URL)

In [5]:
def document_processing(data: str) -> str:
    return re.sub(r'[\r\n\t\f\v]+', '', data)

In [6]:
source_chunks = []
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)

for chunk in splitter.split_text(document):
    source_chunks.append(Document(page_content=chunk, metadata={}))

# всего получилось чанков:
len(source_chunks)

167

In [14]:
# # первый чанк
# print(source_chunks[0])
# page_content = source_chunks[0].page_content
# # длина первого чанка
# print(len(page_content))

In [7]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [8]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

@errors_handler
def get_answer(user_message: str, promt: str, search_index: FAISS, verbose: bool = False, k: int = 5) -> str:
    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(user_message, k=k)

    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    messages = [
        {"role": "system", "content": promt},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией или его название для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{user_message}"}
    ]

    # Компиляция openAI бота:
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)

    if verbose:
        print('\n ===========================================: ')
        print('message_content :\n ======================================== \n', message_content)
        print('\n ===========================================: ')

    return answer


# Пример корректных вопросов в рамках документа:

In [9]:
user_message = "Что такое биологическая безопасность?"
ans = get_answer(user_message, PROMT, db)
ans

' Биологическая безопасность в контексте железнодорожного подвижного состава означает отсутствие возможности возникновения опасного биологического воздействия. Это  означает, что продукция должна быть безопасной для людей и окружающей среды в отношении биологических факторов. Документ указывает, что материалы и вещества,  используемые при производстве железнодорожного подвижного состава и его составных частей, должны быть безопасными с точки зрения биологической безопасности. Это  означает, что они не должны представлять опасности для здоровья людей и окружающей среды в отношении биологических аспектов.'

In [10]:
user_message = "Какими устройствами должны быть оборудованы пассажирские локомотивы?"
ans = get_answer(user_message, PROMT, db)
ans

' Пассажирские локомотивы должны быть оборудованы следующими устройствами: 1. Поездная радиосвязь - для обеспечения связи между локомотивом и другими участками  железнодорожной инфраструктуры. 2. Автоматизированная система управления - обеспечивает контроль скорости движения и возможность получения и передачи речевой информации  при подъездах к входным и выходным светофорам, железнодорожным переездам и станциям. 3. Автоматическая пожарная сигнализация - для обнаружения и предотвращения пожаров в  локомотиве. 4. Регистраторы параметров движения - записывают данные о скорости, ускорении, торможении и других параметрах движения локомотива. 5. Автоматическая  локомотивная сигнализация - предупреждает о приближении к опасным участкам пути, таким как железнодорожные переезды. 6. Электропневматический тормоз - обеспечивает  безопасное торможение локомотива. 7. Установка пожаротушения - предназначена для защиты зон, предусмотренных техническим регламентом, от возгорания. 8. Система  автоматиче

In [11]:
user_message = "В отношении чего проводится анализ состояния производства продукции?"
ans = get_answer(user_message, PROMT, db)
ans

' Анализ состояния производства продукции проводится в отношении следующих аспектов: - Технологических процессов; - Технологической и конструкторской документации (включая  управление ею); - Средств технологического оснащения; - Технологических режимов; - Управления средствами технологического оснащения; - Управления метрологическим  оборудованием; - Методик исследований (испытаний) и измерений; - Порядка проведения контроля сырья и комплектующих изделий; - Порядка проведения контроля продукции в  процессе ее производства; - Управления несоответствующей продукцией; - Порядка работы с рекламациями; - Управления персоналом; - Управления нормативной документацией на  продукцию. Анализ состояния производства продукции проводится с целью установления наличия необходимых условий для изготовления продукции со стабильными  характеристиками, проверяемыми при сертификации. По итогам анализа состояния производства продукции составляется акт о результатах анализа состояния производства  сертифицир

# Пример некорректного вопроса, который выходит за рамки документа:

In [12]:
user_message = "На какую снасть лучше ловить окуня?"
ans = get_answer(user_message, PROMT, db)
ans

' Извините, но я не могу ответить на ваш вопрос о ловле окуня, так как я консультант по техническому регламенту таможенного союза и мои обязанности связаны с  предоставлением информации о безопасности железнодорожного подвижного состава. Если у вас есть вопросы по этой теме, я с удовольствием помогу вам.'